### Cat & Dog classification

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
### vscode 에서 오래 걸릴까봐 코랩에서 돌림

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

In [ ]:
import os
import shutil

if os.path.exists('/content/cats_and_dogs_filtered/'): # 작업 디렉토리는 cats_and_dogs_filtered

    shutil.rmtree('/content/cats_and_dogs_filtered')
    print('/content/cats_and_dogs_filtered/ is removed !!!')

In [ ]:
import zipfile

with zipfile.ZipFile('/content/cats_and_dogs_filtered.zip', 'r') as target_file:
    target_file.extractall('/content/cats_and_dogs_filtered/')

In [ ]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [ ]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

base_model.summary()

In [ ]:
model = Sequential()

model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(2, activation='softmax')) # 예측 결과가 2개(cat, dog)이므로 최종출력은 2

model.summary()

In [3]:
train_data_gen = ImageDataGenerator(rescale=1./255, # 정규화
                                    rotation_range=10, width_shift_range=0.1,
                                    height_shift_range=0.1, shear_range=0.1, zoom_range=0.1)

test_data_gen = ImageDataGenerator(rescale=1./255) # 정규화

In [ ]:
train_dir = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/train'

test_dir = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/validation'

In [ ]:
train_data = train_data_gen.flow_from_directory(train_dir, batch_size=32,
                                                color_mode='rgb', shuffle=True, class_mode='categorical',
                                                target_size=(IMG_WIDTH, IMG_HEIGHT))

test_data = test_data_gen.flow_from_directory(test_dir, batch_size=32,
                                              color_mode='rgb', shuffle=True, class_mode='categorical',
                                              target_size=(IMG_WIDTH, IMG_HEIGHT))

In [ ]:
print(train_data.class_indices.items())
print(test_data.class_indices.items())

print(len(train_data.classes))
print(len(test_data.classes))

print(train_data.num_classes)
print(test_data.num_classes)

NameError: name 'train_data' is not defined

In [ ]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.00002), metrics=['accuracy'])

In [ ]:
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

save_file_name = './cats_and_dogs_filtered_Xception_Colab.h5'

checkpoint = ModelCheckpoint(save_file_name,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')

earlystopping = EarlyStopping(monitor='val_loss',
                              patience=5)

start_time = datetime.now()

hist = model.fit(train_data, epochs=30,
                 validation_data=test_data,
                 callbacks=[checkpoint, earlystopping])

In [ ]:
print(f"Min Validation Loss : {min(hist.history['val_loss'])}")
print(f"Max Validation Accuracy : {max(hist.history['val_accuracy'])}")

In [ ]:
import matplotlib.pyplot as plt

plt.title('accuracy trend')
plt.grid()
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='validation')
plt.legend(loc='best')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.title('loss trend')
plt.grid()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='validation')
plt.legend(loc='best')
plt.show()

In [ ]:
# 코랩에서 구글드라이브 연동전에 해야함.
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import cv2
import glob

test_img_name_list = glob.glob('/content/drive/MyDrive/cat_dog/*')

for i in range(len(test_img_name_list)):
    # 이미지 로드 및 전처리
    src_img = cv2.imread(test_img_name_list[i], cv2.IMREAD_COLOR)
    src_img = cv2.resize(src_img, dsize=(IMG_WIDTH, IMG_HEIGHT))
    dst_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
    dst_img = dst_img / 255.0
    
    # 전처리된 이미지로 덮어쓰기
    test_img_name_list[i] = dst_img
    
print(len(test_img_name_list))

In [ ]:
# 이미지 리스트 출력
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.axis('off')
    plt.imshow(test_img_name_list[i])
plt.show()

# 이미지 shape 확인
print(test_img_name_list[0].shape)

In [ ]:
import numpy as np
print(np.array(test_img_name_list).shape)

# 모델 예측
class_name = ['cat', 'dog']
pred = model.predict(np.array(test_img_name_list))
print(pred.shape)
print(pred)

In [ ]:
# 예측 결과 시각화
plt.figure(figsize=(12, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    prediction = str(class_name[np.argmax(pred[i])])
    probility = '{0:0.2f}'.format(100*max(pred[i]))
    title_str = prediction + ', ' + probility + '%'
    plt.axis('off')
    plt.title(title_str)
    plt.imshow(test_img_name_list[i])
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# 모델 불러오기
model = load_model('/content/cats_and_dogs_filtered_Xception_Colab.h5')

# 이미지 경로 지정
image_path = '/content/다운로드.jpg'

# 이미지 불러오기 및 전처리
image = load_img(image_path, target_size=(224,224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = image / 255.0 # 이미지 정규화

# 예측 수행
class_name = ['cat', 'dog']
predictions = model.predict(image)
predicted_class = str(class_name[np.argmax(predictions)])

# 예측 결과 출력
print(f'Predicted class : {predicted_class}')